<a href="https://colab.research.google.com/github/SasinduShanaka/heart_disease_predict_model/blob/main/Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
df = pd.read_csv("heart_disease.csv")

df.head()


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140.0,289.0,0.0,Normal,172.0,N,0.0,Up,0
1,49,F,NAP,160.0,180.0,0.0,Normal,156.0,N,1.0,Flat,1
2,37,M,ATA,130.0,283.0,0.0,ST,98.0,N,0.0,Up,0
3,48,F,ASY,138.0,214.0,0.0,Normal,108.0,Y,1.5,Flat,1
4,54,M,NAP,150.0,195.0,0.0,Normal,122.0,N,0.0,Up,0


In [4]:
df.shape

(922, 12)

In [5]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,922.000000,920.000000,918.000000,921.000000,919.000000,920.000000,922.000000
mean,54.244035,134.192391,202.845316,0.233442,136.769314,0.888587,0.555315
std,21.265503,48.054115,150.037800,0.423251,25.467932,1.067112,0.497201
min,3.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,175.000000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,155.500000,1.500000,1.000000
max,622.000000,1300.000000,3331.000000,1.000000,202.000000,6.200000,1.000000


In [6]:
df.isnull().sum()

,0
Age,0
Sex,0
ChestPainType,4
RestingBP,2
Cholesterol,4
FastingBS,1
RestingECG,0
MaxHR,3
ExerciseAngina,0
Oldpeak,2


In [8]:
from sklearn.impute import SimpleImputer

num_cols = df.select_dtypes(include=["int64","float64"]).columns
cat_cols = df.select_dtypes(include=["object"]).columns

df[num_cols] = SimpleImputer(strategy="median").fit_transform(df[num_cols])
df[cat_cols] = SimpleImputer(strategy="most_frequent").fit_transform(df[cat_cols])




In [12]:
df.duplicated().sum()

np.int64(0)

In [11]:
df.drop_duplicates(inplace=True)

In [ ]:
import numpy as np

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.clip(df[col], lower, upper)
